In [1]:
import math
import time
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def load_data(path):
    df = pd.read_csv(path)
    data = df.drop(columns=['Unnamed: 0'])
    
    return data

In [3]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    return trainX, trainY, testX, testY

In [4]:
def getTestY(data):
    tmp_scaler = StandardScaler()
    tmp_data = tmp_scaler.fit_transform(data)
    tmp_data = tmp_scaler.inverse_transform(tmp_data)
    _, _, _, testY = preprocess_data(tmp_data, data.shape[0], train_rate, seq_len, pre_len)
    testY = np.array(testY)
    testY = np.reshape(testY, [-1, pre_len])
    
    return testY

In [5]:
 def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    mape = mean_absolute_percentage_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    
    return rmse, mae, mape, 1-F_norm

In [6]:
def predictSVR(data, testY):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    rmses, maes, mapes, accs = [], [], [], []
    for i in range(data.shape[1]):
        print('Node', i)
        start = time.time()
        a = data[:, i]
        aX, aY, tX, tY = preprocess_data(a, data.shape[0], train_rate, seq_len, pre_len)
        
        aX = np.array(aX)
        aX = np.reshape(aX, [-1, seq_len])
        aY = np.array(aY)
        aY = np.reshape(aY, [-1, pre_len])

        tX = np.array(tX)
        tX = np.reshape(tX, [-1, seq_len])
        tY = np.array(tY)
        tY = np.reshape(tY, [-1, pre_len])
        
        print('(', aX.shape, aY.shape, tX.shape, tY.shape, round(aX.mean(),3), ')')
        reg = MultiOutputRegressor(SVR(kernel='linear'))
        reg.fit(aX, aY)
        pred = reg.predict(tX)
        
        mean = scaler.mean_[i]
        std = np.sqrt(scaler.var_[i])
        pred = pred*std + mean
        tY = tY*std + mean
        if i==data.shape[1]-1: tY = testY[tY.shape[0]*i:]
        else: tY = testY[tY.shape[0] * i:tY.shape[0] * (i+1)]
        
        rmse, mae, mape, acc = evaluation(tY, pred)
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        accs.append(acc)
        
        print('(', rmse, mae, mape, acc, time.time() - start)
    
    print('RMSE: ' + str(np.mean(rmses)) + ', MAE: ' + str(np.mean(maes)) + ', MAPE: ' + str(np.mean(mapes)) + ', ACC: ' + str(np.mean(accs)))

In [7]:
train_rate = 0.8
seq_len = 12
pre_len = 3

In [8]:
path_0 = 'data/METR-LA/speed_la_0.csv'
path_5 = 'data/METR-LA/speed_la_5.csv'
path_10 = 'data/METR-LA/speed_la_10.csv'
path_20 = 'data/METR-LA/speed_la_20.csv'

In [9]:
masterData = load_data(path_0)
print(masterData.shape, (masterData == 0).sum().sum())
testY = getTestY(masterData)
testY.shape, (testY == 0).sum().sum()

(34272, 207) 0


((1415880, 3), 0)

### Unobserved Nodes = 10%

In [10]:
data = load_data(path_10)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(34272, 207) 685440
Node 0
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.011 )
( 21.616228450057317 13.926198553368323 0.5244862461988762 0.6230256136447425 113.94285869598389
Node 1
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) -0.008 )
( 18.26648352613412 11.063702558573487 0.4569673765743712 0.680020572525661 269.10967326164246
Node 2
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) -0.016 )
( 13.85381596504673 8.937841997846277 0.1951002449352074 0.77211665931794 106.52243542671204
Node 3
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.012 )
( 17.455409363727217 13.544618284371245 0.37697515100663187 0.6933929486916033 247.51571702957153
Node 4
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.038 )
( 24.71941520058156 20.180752960211652 0.661568745992149 0.5332797321727858 143.99981212615967
Node 5
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.037 )
( 15.28920189826233 12.297650889168802 0.20930376865902764 0.7581496306267008 275.39015913009644
Node 6
( (27402, 12) (27402, 3) (6840

( 14.28329600166949 9.48458812399416 0.1692847570760743 0.7780665951430219 172.21395468711853
Node 55
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.001 )
( 11.44980811750016 7.762543200366871 0.1890530533832361 0.8166395701985154 284.5810785293579
Node 56
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) -0.156 )
( 23.702479719687947 18.370283514107424 0.3157155396709492 0.6261116831832021 190.68783688545227
Node 57
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.005 )
( 14.68570580412888 10.743295366084118 0.181589038309471 0.7708127824739556 129.7345106601715
Node 58
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.011 )
( 10.085806547076952 6.954632965400063 0.14721321294926407 0.8423236836162786 222.99954390525818
Node 59
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 62.94156676558993 62.62776348664866 1.0 0.0 0.08095932006835938
Node 60
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.089 )
( 20.729664888403295 15.851785651581231 0.2595704001110028 0.6626965012526389 129.26658

( 24.295236556982864 16.27858917548802 0.6709564610627704 0.5594569684856894 234.37370324134827
Node 108
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) -0.006 )
( 23.48735487340049 15.81202618177 0.6084429631454804 0.5776430610564809 193.46553301811218
Node 109
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) -0.017 )
( 15.48496449269197 11.330645722729463 0.3238391245708104 0.7412534082976304 287.2177984714508
Node 110
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) -0.013 )
( 13.32607708292465 7.536184219277657 0.2208163259517558 0.78618968982679 203.19556832313538
Node 111
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.004 )
( 17.406854280451796 10.061056306055681 0.2550948474443836 0.7212396418626885 111.70232510566711
Node 112
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.004 )
( 16.945480648364615 12.605353062505564 0.27371923849875407 0.7286074900464129 217.42083525657654
Node 113
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.006 )
( 17.81099478226342 11.724327725951719 0.2650910

( 30.09891984895459 23.716570155649674 0.7737736109329968 0.40117921027137715 80.96449375152588
Node 161
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.001 )
( 24.80096462314435 17.312425900272228 0.7429585524251507 0.5395115884282842 127.25987911224365
Node 162
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.016 )
( 18.2219746704714 10.900300854323651 0.2798344236847326 0.7025960756248311 89.69931197166443
Node 163
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.036 )
( 30.021686643560518 22.207590073188566 0.42573391139369904 0.5136881296237702 56.336482763290405
Node 164
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 62.01667929829802 61.45448116819838 1.0 0.0 0.08858537673950195
Node 165
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.018 )
( 20.080967037026625 14.326726728003335 0.2592454650473287 0.6758614749235373 203.7754089832306
Node 166
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.004 )
( 12.726664479723027 8.149874652064597 0.19540208421840932 0.7989474513161721 

### Unobserved Node = 20%

In [11]:
data = load_data(path_20)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(34272, 207) 1405152
Node 0
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 57.34137180791473 54.98241282217899 1.0 0.0 0.06316208839416504
Node 1
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 57.08643105688098 55.0258652998236 1.0 0.0 0.0621340274810791
Node 2
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 60.793456527282395 60.10709553590761 1.0 0.0 0.061110734939575195
Node 3
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.012 )
( 17.455409363727217 13.544618284371245 0.37697515100663187 0.6933929486916033 248.01901960372925
Node 4
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.038 )
( 24.71941520058156 20.180752960211652 0.661568745992149 0.5332797321727858 143.7003755569458
Node 5
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.037 )
( 15.28920189826233 12.297650889168802 0.20930376865902764 0.7581496306267008 275.3524878025055
Node 6
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.029 )
( 13.975189822409867 9.292393398729104 0.15299835248013213 0.77755016426

( 12.675585158098789 11.231111974352373 0.18842010769935688 0.8065364801650353 286.97404980659485
Node 54
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.017 )
( 14.28329600166949 9.48458812399416 0.1692847570760743 0.7780665951430219 172.4350230693817
Node 55
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.001 )
( 11.44980811750016 7.762543200366871 0.1890530533832361 0.8166395701985154 286.0115370750427
Node 56
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) -0.156 )
( 23.702479719687947 18.370283514107424 0.3157155396709492 0.6261116831832021 190.43235635757446
Node 57
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.005 )
( 14.68570580412888 10.743295366084118 0.181589038309471 0.7708127824739556 130.43591022491455
Node 58
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 63.96525983351927 63.503393669358694 1.0 0.0 0.06798195838928223
Node 59
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 62.94156676558993 62.62776348664866 1.0 0.0 0.06670546531677246
Node 60
( (27402, 12)

( 24.295236556982864 16.27858917548802 0.6709564610627704 0.5594569684856894 236.5830991268158
Node 108
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 55.61020243245347 52.75135069541142 1.0 0.0 0.08464980125427246
Node 109
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) -0.017 )
( 15.48496449269197 11.330645722729463 0.3238391245708104 0.7412534082976304 287.55928206443787
Node 110
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 62.3266346329556 61.226330293325766 1.0 0.0 0.08669257164001465
Node 111
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 62.44379364686218 61.34131045205592 1.0 0.0 0.08661890029907227
Node 112
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.004 )
( 16.945480648364615 12.605353062505564 0.27371923849875407 0.7286074900464129 218.55861735343933
Node 113
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.006 )
( 17.81099478226342 11.724327725951719 0.2650910621234608 0.714732705003654 172.47837471961975
Node 114
( (27402, 12) (27402, 3) (6840, 12) 

( 18.2219746704714 10.900300854323651 0.2798344236847326 0.7025960756248311 90.33307766914368
Node 163
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.036 )
( 30.021686643560518 22.207590073188566 0.42573391139369904 0.5136881296237702 57.36364006996155
Node 164
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.008 )
( 17.063651401543442 11.359937660241982 0.2131043315963929 0.724853836183845 137.640070438385
Node 165
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.0 )
( 61.95180604431264 61.397548114112475 1.0 0.0 0.08378481864929199
Node 166
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.004 )
( 12.726664479723027 8.149874652064597 0.19540208421840932 0.7989474513161721 120.74261498451233
Node 167
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) -0.009 )
( 16.573865387756438 9.660579882008424 0.32011339023966007 0.7243863515084075 210.11354064941406
Node 168
( (27402, 12) (27402, 3) (6840, 12) (6840, 3) 0.033 )
( 20.13819256083765 13.10847347998235 0.46117515647145657 0.6501149069604026 